In [40]:
import pandas as pd
import numpy as np

In [29]:
df = pd.read_csv('D:\study tool\Programming & Data Science Study Materials\Dat Science and AI\Jupyter Notebook files\model1_data (1).csv')

In [31]:
df.head(1)

,Unnamed: 0.1,Unnamed: 0,loan_len,Story,Image,purpose_Beauty Salon,purpose_Cattle,purpose_Clothing Sales,purpose_Dairy,purpose_Farming,...,purpose_Grocery Store,purpose_Home Products Sales,purpose_Others,purpose_Restaurant,purpose_Retail,purpose_Services,purpose_Shoe Sales,purpose_Tailoring,expected,Emotion
0,0,0,12,This business is a member of the Golden Crown ...,https://www.kiva.org/img/w960/620df2a0f8498acb...,True,False,False,False,False,...,False,False,False,False,False,False,False,False,575,Happy


In [56]:
df_img = pd.DataFrame(df['Image'])
df_story = pd.DataFrame(df['Story'])

In [39]:
df_story.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6657 entries, 0 to 6656
Data columns (total 1 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   Story   5391 non-null   object
dtypes: object(1)
memory usage: 52.1+ KB


text context

In [34]:
from gensim.utils import simple_preprocess
from gensim.models import Word2Vec

In [49]:
df_story=pd.DataFrame(df_story['Story'].astype('str','utf-8'))
df_story

,Story
0,This business is a member of the Golden Crown ...
1,This business is a member of the Golden Crown ...
2,This business is a member of the Golden Crown ...
3,This business is a member of the Golden Crown ...
4,I am the owner of my own beauty salon. I had ...
...,...
6652,Doña Antonia is from San Luis Potosí. She has ...
6653,Maria is married and has a 6 year-old son. Her...
6654,Lorenza is a widow and has four married childr...
6655,"Erika is from Tamaulipas, she is married and h..."


In [51]:
df_story['tokenized_context']= df_story['Story'].apply(lambda text:simple_preprocess(text))

In [52]:
df_story

,Story,tokenized_context
0,This business is a member of the Golden Crown ...,"[this, business, is, member, of, the, golden, ..."
1,This business is a member of the Golden Crown ...,"[this, business, is, member, of, the, golden, ..."
2,This business is a member of the Golden Crown ...,"[this, business, is, member, of, the, golden, ..."
3,This business is a member of the Golden Crown ...,"[this, business, is, member, of, the, golden, ..."
4,I am the owner of my own beauty salon. I had ...,"[am, the, owner, of, my, own, beauty, salon, h..."
...,...,...
6652,Doña Antonia is from San Luis Potosí. She has ...,"[doña, antonia, is, from, san, luis, potosí, s..."
6653,Maria is married and has a 6 year-old son. Her...,"[maria, is, married, and, has, year, old, son,..."
6654,Lorenza is a widow and has four married childr...,"[lorenza, is, widow, and, has, four, married, ..."
6655,"Erika is from Tamaulipas, she is married and h...","[erika, is, from, tamaulipas, she, is, married..."


In [53]:
# Train Word2Vec model
model = Word2Vec(sentences=df_story['tokenized_context'], vector_size=150, window=5, min_count=1, workers=4)

In [54]:
def document_to_vector(tokens, model):
    # Get the vectors for each token that exists in the model's vocabulary
    vectors = [model.wv[word] for word in tokens if word in model.wv]
    
    # If the document has words in the vocabulary, return the mean of the vectors, else return a zero vector
    if vectors:
        return np.mean(vectors, axis=0)
    else:
        return np.zeros(model.vector_size)

# Apply the function to each row in the DataFrame
df['text_vector'] = df_story['tokenized_context'].apply(lambda x: document_to_vector(x, model))

In [55]:
df['text_vector'].sample(2)

3860    [0.9462125, 0.13018529, -0.33458236, 1.192315,...
3120    [0.26501614, -0.2458195, -0.3404847, 0.6811321...
Name: text_vector, dtype: object

Image Data

In [57]:
df_img.sample(1)

,Image
4886,https://www.kiva.org/img/w960/dc2855b09f45987c...


In [58]:
import tensorflow as tf
from tensorflow.keras.applications import VGG16
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.vgg16 import preprocess_input
import numpy as np
import requests
from PIL import Image
from io import BytesIO

In [59]:
# Load the VGG16 model without the top (fully connected) layers
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Add a global average pooling layer and a dense layer to get a 128-dimensional vector
x = base_model.output
x = tf.keras.layers.GlobalAveragePooling2D()(x)
x = Dense(128, activation='relu')(x)

# Define the new model
model = Model(inputs=base_model.input, outputs=x)

In [60]:
def load_and_preprocess_image(img_url):
    try:
        response = requests.get(img_url, timeout=10)
        response.raise_for_status()
        img = Image.open(BytesIO(response.content))

        # Convert RGBA to RGB if necessary
        if img.mode == 'RGBA':
            img = img.convert('RGB')

        img = img.resize((224, 224))
        img_array = np.array(img)
        img_array = np.expand_dims(img_array, axis=0)
        img_array = preprocess_input(img_array)
        return img_array

    except requests.exceptions.RequestException as e:
        print(f"Error loading image {img_url}: {e}")
        return None
    except Exception as e:
        print(f"Error processing image {img_url}: {e}")
        return None


In [61]:
# Function to extract features using VGG16 and convert them into context vectors
def extract_features(image_url):
    img = load_and_preprocess_image(image_url)
    if img is not None:
        features = model.predict(img)
        context_vector = features.flatten()
        return context_vector
    else:
        return np.zeros(128)  # If image loading fails, return a zero vector of appropriate size

# Apply the function to each row in the DataFrame
df['img_context_vector'] = df_img['Image'].apply(extract_features)

Error loading image https://www.kiva.org/img/w960/620df2a0f8498acb990e9c65c4c2aca3.jpg: ('Connection aborted.', ConnectionResetError(10054, 'An existing connection was forcibly closed by the remote host', None, 10054, None))
Error loading image https://www.kiva.org/img/w960/2bd1f94b704885036c7d431b24e5d6e2.jpg: ('Connection aborted.', ConnectionResetError(10054, 'An existing connection was forcibly closed by the remote host', None, 10054, None))
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 588ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 445ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 440ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 511ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 595ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 696ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 402ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 846ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 435ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 600ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 532ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 457ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 482ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 523ms/step
1/1 

KeyboardInterrupt: 

In [4]:
import tensorflow as tf
print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))

Num GPUs Available:  0


In [2]:
import tensorflow as tf
print(tf.__version__)

2.16.2


In [4]:
import tensorflow as tf

# Check if GPU is available
if tf.config.list_physical_devices('GPU'):
    device = 'GPU'
else:
    device = 'CPU'

print(f'Device selected: {device}')

Device selected: CPU


Niche kuchh nahi h

In [1]:
import re

# Original text
text = """Faleminderit per zemërgjerësinë gjithësecilit që dha donacion! Po e mbyllim këtë GofundMe me shumën $22,725———————————————————————-Të dashur miq dhe njerëz me zemër të madhe,Unë jam Fjoralba Ruça, një mësuese dhe nënë krenare (prind i vetëm) e dy fëmijëve të dashur, Gentonit 8 vjeç dhe Pegit 15 vjeç. Në mars të këtij viti, jeta ime mori një kthesë dramatike kur mësova se kisha kancer në stomak. Pas operacionit të vështirë në Tiranë për heqjen e plotë të stomakut, tani jam në Ankara, Turqi. Jam larg fëmijëve të mi dhe kjo është një dhimbje që s'e përshkruaj dot me fjalë.  Çdo ditë është një betejë. Çdo seancë kimioterapie është një hap drejt shërimit, por edhe një kujtesë e dhimbshme e luftës që po përballoj. Kam përfunduar 5 seanca nga dy ciklet e planifikuara, dhe më duhet të vazhdoj këtë rrugë për të pasur një shans për t'u rikthyer pranë Gentonit dhe Pegit, të cilët më presin me padurim në shtëpi. Ndihma e miqve dhe familjes më ka mbajtur deri tani, por shpenzimet janë të mëdha dhe të vazhdueshme. Trajtimi, akomodimi dhe ushqimet speciale janë të domosdoshme, por përtej mundësive të mia financiare. Për këtë arsye, po ju kërkoj ndihmë me zemër në dorë. E di që bota është plot me njerëz që përballen me sfida, por nëse zemra juaj e ndjen të më ndihmojë, do të isha pafundësisht mirënjohëse. Çdo ndihmë që mund të jepni është një dhuratë e çmuar që do të më afrojë më pranë familjes sime dhe shpresës për të ardhmen. Çdo mbështetje është një rreze shprese dhe një kujtim i dashurisë dhe solidaritetit që mund të bëjë mrekulli. Ju lutem, bëhuni pjesë e kësaj lufte, ndihmoni që të kthehem pranë atyre që më duan më shumë se çdo gjë tjetër në botë. Me mirënjohje dhe shpresë, \xa0Fjoralba Ruça ——————————————————————————-Dear friends and kind-hearted people, I am Fjoralba Ruça, a teacher and a proud mother (single mother) of two beloved children, 8-year-old Gentoni and 15-year-old Pegi. In March of this year, my life took a dramatic turn when I learned I had stomach cancer. After a difficult surgery in Tirana to remove my entire stomach, I am now in Ankara, Turkey. Being away from my children is a pain I cannot put into words.  Every day is a battle. Each chemotherapy session is a step towards healing, but also a painful reminder of the struggle I am facing. I have completed 5 sessions out of two planned cycles, and I must continue on this path to have a chance of returning to Gentoni and Pegi, who are eagerly waiting for me at home. The support of friends and family has sustained me so far, but the expenses are significant and ongoing. Treatment, accommodation, and special dietary needs are essential but beyond my financial means. For this reason, I am reaching out to you with my heart in my hand. I know the world is full of people facing challenges, but if your heart feels moved to help me, I would be endlessly grateful. Any assistance you can provide is a precious gift that will bring me closer to my family and to hope for the future. Every bit of support is a ray of hope and a reminder of the love and solidarity that can create miracles. Please, join this fight, help me return to those who love me more than anything else in the world. \xa0With gratitude and hope, \xa0Fjoralba RuçaRead more"""

# Regular expression to filter out non-English text
clean_text = re.sub(r'[^\x00-\x7F]+', '', text)

# Output the cleaned text
print(clean_text)


Faleminderit per zemrgjersin gjithsecilit q dha donacion! Po e mbyllim kt GofundMe me shumn $22,725-T dashur miq dhe njerz me zemr t madhe,Un jam Fjoralba Rua, nj msuese dhe nn krenare (prind i vetm) e dy fmijve t dashur, Gentonit 8 vje dhe Pegit 15 vje. N mars t ktij viti, jeta ime mori nj kthes dramatike kur msova se kisha kancer n stomak. Pas operacionit t vshtir n Tiran pr heqjen e plot t stomakut, tani jam n Ankara, Turqi. Jam larg fmijve t mi dhe kjo sht nj dhimbje q s'e prshkruaj dot me fjal.  do dit sht nj betej. do seanc kimioterapie sht nj hap drejt shrimit, por edhe nj kujtes e dhimbshme e lufts q po prballoj. Kam prfunduar 5 seanca nga dy ciklet e planifikuara, dhe m duhet t vazhdoj kt rrug pr t pasur nj shans pr t'u rikthyer pran Gentonit dhe Pegit, t cilt m presin me padurim n shtpi. Ndihma e miqve dhe familjes m ka mbajtur deri tani, por shpenzimet jan t mdha dhe t vazhdueshme. Trajtimi, akomodimi dhe ushqimet speciale jan t domosdoshme, por prtej mundsive t mia financia

In [17]:
import nltk
# nltk.download('words')
words = set(nltk.corpus.words.words())

sent = "Faleminderit per zemrgjersin gjithsecilit q dha donacion! Po e mbyllim kt GofundMe me shumn $22,725-T dashur miq dhe njerz me zemr t madhe,Un jam Fjoralba Rua, nj msuese dhe nn krenare (prind i vetm) e dy fmijve t dashur, Gentonit 8 vje dhe Pegit 15 vje. N mars t ktij viti, jeta ime mori nj kthes dramatike kur msova se kisha kancer n stomak. Pas operacionit t vshtir n Tiran pr heqjen e plot t stomakut, tani jam n Ankara, Turqi. Jam larg fmijve t mi dhe kjo sht nj dhimbje q s'e prshkruaj dot me fjal.  do dit sht nj betej. do seanc kimioterapie sht nj hap drejt shrimit, por edhe nj kujtes e dhimbshme e lufts q po prballoj. Kam prfunduar 5 seanca nga dy ciklet e planifikuara, dhe m duhet t vazhdoj kt rrug pr t pasur nj shans pr t'u rikthyer pran Gentonit dhe Pegit, t cilt m presin me padurim n shtpi. Ndihma e miqve dhe familjes m ka mbajtur deri tani, por shpenzimet jan t mdha dhe t vazhdueshme. Trajtimi, akomodimi dhe ushqimet speciale jan t domosdoshme, por prtej mundsive t mia financiare. Pr kt arsye, po ju krkoj ndihm me zemr n dor. E di q bota sht plot me njerz q prballen me sfida, por nse zemra juaj e ndjen t m ndihmoj, do t isha pafundsisht mirnjohse. do ndihm q mund t jepni sht nj dhurat e muar q do t m afroj m pran familjes sime dhe shpress pr t ardhmen. do mbshtetje sht nj rreze shprese dhe nj kujtim i dashuris dhe solidaritetit q mund t bj mrekulli. Ju lutem, bhuni pjes e ksaj lufte, ndihmoni q t kthehem pran atyre q m duan m shum se do gj tjetr n bot. Me mirnjohje dhe shpres, Fjoralba Rua -Dear friends and kind-hearted people, I am Fjoralba Rua, a teacher and a proud mother (single mother) of two beloved children, 8-year-old Gentoni and 15-year-old Pegi. In March of this year, my life took a dramatic turn when I learned I had stomach cancer. After a difficult surgery in Tirana to remove my entire stomach, I am now in Ankara, Turkey. Being away from my children is a pain I cannot put into words.  Every day is a battle. Each chemotherapy session is a step towards healing, but also a painful reminder of the struggle I am facing. I have completed 5 sessions out of two planned cycles, and I must continue on this path to have a chance of returning to Gentoni and Pegi, who are eagerly waiting for me at home. The support of friends and family has sustained me so far, but the expenses are significant and ongoing. Treatment, accommodation, and special dietary needs are essential but beyond my financial means. For this reason, I am reaching out to you with my heart in my hand. I know the world is full of people facing challenges, but if your heart feels moved to help me, I would be endlessly grateful. Any assistance you can provide is a precious gift that will bring me closer to my family and to hope for the future. Every bit of support is a ray of hope and a reminder of the love and solidarity that can create miracles. Please, join this fight, help me return to those who love me more than anything else in the world. With gratitude and hope, Fjoralba RuaRead more"

" ".join(w for w in nltk.wordpunct_tokenize(sent) 
         if w.lower() in words and w.lower()!=re.compile(r"[b-z]"))

'per q dha Po e me T me t Un jam i e t N t se n t n e plot t jam n Jam t mi q s e dot me do dit do hap e e q po e m t t t u t m me n e m ka t t t t po me n dor E di q bota plot me q me e t m do t do q mund t e q do t m m sime t do i q mund t e q t q m m se do n bot Me Dear and kind hearted people I am a teacher and a mother single mother of two beloved year old and year old In March of this year my life took a dramatic turn when I learned I had stomach cancer After a difficult surgery in to remove my entire stomach I am now in Turkey Being away from my is a pain I cannot put into Every day is a battle Each chemotherapy session is a step towards healing but also a painful reminder of the struggle I am facing I have sessions out of two and I must continue on this path to have a chance of to and who are eagerly waiting for me at home The support of and family sustained me so far but the are significant and ongoing Treatment accommodation and special dietary needs are essential but beyond 

In [26]:
import nltk
import re

# Load the NLTK words corpus into a set for fast lookup
words = set(nltk.corpus.words.words())

# Input text
sent = """Faleminderit per zemrgjersin gjithsecilit q dha donacion! Po e mbyllim kt GofundMe me shumn $22,725-T dashur miq dhe njerz me zemr t madhe,Un jam Fjoralba Rua, nj msuese dhe nn krenare (prind i vetm) e dy fmijve t dashur, Gentonit 8 vje dhe Pegit 15 vje. N mars t ktij viti, jeta ime mori nj kthes dramatike kur msova se kisha kancer n stomak. Pas operacionit t vshtir n Tiran pr heqjen e plot t stomakut, tani jam n Ankara, Turqi. Jam larg fmijve t mi dhe kjo sht nj dhimbje q s'e prshkruaj dot me fjal.  do dit sht nj betej. do seanc kimioterapie sht nj hap drejt shrimit, por edhe nj kujtes e dhimbshme e lufts q po prballoj. Kam prfunduar 5 seanca nga dy ciklet e planifikuara, dhe m duhet t vazhdoj kt rrug pr t pasur nj shans pr t'u rikthyer pran Gentonit dhe Pegit, t cilt m presin me padurim n shtpi. Ndihma e miqve dhe familjes m ka mbajtur deri tani, por shpenzimet jan t mdha dhe t vazhdueshme. Trajtimi, akomodimi dhe ushqimet speciale jan t domosdoshme, por prtej mundsive t mia financiare. Pr kt arsye, po ju krkoj ndihm me zemr n dor. E di q bota sht plot me njerz q prballen me sfida, por nse zemra juaj e ndjen t m ndihmoj, do t isha pafundsisht mirnjohse. do ndihm q mund t jepni sht nj dhurat e muar q do t m afroj m pran familjes sime dhe shpress pr t ardhmen. do mbshtetje sht nj rreze shprese dhe nj kujtim i dashuris dhe solidaritetit q mund t bj mrekulli. Ju lutem, bhuni pjes e ksaj lufte, ndihmoni q t kthehem pran atyre q m duan m shum se do gj tjetr n bot. Me mirnjohje dhe shpres, Fjoralba Rua -Dear friends and kind-hearted people, I am Fjoralba Rua, a teacher and a proud mother (single mother) of two beloved children, 8-year-old Gentoni and 15-year-old Pegi. In March of this year, my life took a dramatic turn when I learned I had stomach cancer. After a difficult surgery in Tirana to remove my entire stomach, I am now in Ankara, Turkey. Being away from my children is a pain I cannot put into words.  Every day is a battle. Each chemotherapy session is a step towards healing, but also a painful reminder of the struggle I am facing. I have completed 5 sessions out of two planned cycles, and I must continue on this path to have a chance of returning to Gentoni and Pegi, who are eagerly waiting for me at home. The support of friends and family has sustained me so far, but the expenses are significant and ongoing. Treatment, accommodation, and special dietary needs are essential but beyond my financial means. For this reason, I am reaching out to you with my heart in my hand. I know the world is full of people facing challenges, but if your heart feels moved to help me, I would be endlessly grateful. Any assistance you can provide is a precious gift that will bring me closer to my family and to hope for the future. Every bit of support is a ray of hope and a reminder of the love and solidarity that can create miracles. Please, join this fight, help me return to those who love me more than anything else in the world. With gratitude and hope, Fjoralba Rua"""

# Function to check if a word is a valid word and not a single-letter word
def is_valid_word(word):
    return word.lower()=='i' or (len(word) > 1 and word.lower() in words)

# Tokenize the sentence and filter out valid words
filtered_words = [w for w in nltk.wordpunct_tokenize(sent) if is_valid_word(w)]

# Join the filtered words into a single string
filtered_text = " ".join(filtered_words)

print(filtered_text)

per dha Po me me Un jam i se plot jam Jam mi dot me do dit do hap po me ka po me dor di bota plot me me do do mund do sime do i mund se do bot Me Dear and kind hearted people I am teacher and mother single mother of two beloved year old and year old In March of this year my life took dramatic turn when I learned I had stomach cancer After difficult surgery in to remove my entire stomach I am now in Turkey Being away from my is pain I cannot put into Every day is battle Each chemotherapy session is step towards healing but also painful reminder of the struggle I am facing I have sessions out of two and I must continue on this path to have chance of to and who are eagerly waiting for me at home The support of and family sustained me so far but the are significant and ongoing Treatment accommodation and special dietary needs are essential but beyond my financial For this reason I am reaching out to you with my heart in my hand I know the world is full of people facing but if your heart to

In [25]:
def clean_text_until_delimiter(text, delimiter):
    # Split the text by the delimiter
    parts = text.split(delimiter)
    
    # Take the part of the text before the delimiter
    clean_part = parts[0].strip()
    
    return clean_part

# Example text
text = """Faleminderit per zemërgjerësinë gjithësecilit që dha donacion! Po e mbyllim këtë GofundMe me shumën $22,725——————————————————————————Të dashur miq dhe njerëz me zemër të madhe,Unë jam Fjoralba Ruça, një mësuese dhe nënë krenare (prind i vetëm) e dy fëmijëve të dashur, Gentonit 8 vjeç dhe Pegit 15 vjeç. Në mars të këtij viti, jeta ime mori një kthesë dramatike kur mësova se kisha kancer në stomak. Pas operacionit të vështirë në Tiranë për heqjen e plotë të stomakut, tani jam në Ankara, Turqi. Jam larg fëmijëve të mi dhe kjo është një dhimbje që s'e përshkruaj dot me fjalë.  Çdo ditë është një betejë. Çdo seancë kimioterapie është një hap drejt shërimit, por edhe një kujtesë e dhimbshme e luftës që po përballoj. Kam përfunduar 5 seanca nga dy ciklet e planifikuara, dhe më duhet të vazhdoj këtë rrugë për të pasur një shans për t'u rikthyer pranë Gentonit dhe Pegit, të cilët më presin me padurim në shtëpi. Ndihma e miqve dhe familjes më ka mbajtur deri tani, por shpenzimet janë të mëdha dhe të vazhdueshme. Trajtimi, akomodimi dhe ushqimet speciale janë të domosdoshme, por përtej mundësive të mia financiare. Për këtë arsye, po ju kërkoj ndihmë me zemër në dorë. E di që bota është plot me njerëz që përballen me sfida, por nëse zemra juaj e ndjen të më ndihmojë, do të isha pafundësisht mirënjohëse. Çdo ndihmë që mund të jepni është një dhuratë e çmuar që do të më afrojë më pranë familjes sime dhe shpresës për të ardhmen. Çdo mbështetje është një rreze shprese dhe një kujtim i dashurisë dhe solidaritetit që mund të bëjë mrekulli. Ju lutem, bëhuni pjesë e kësaj lufte, ndihmoni që të kthehem pranë atyre që më duan më shumë se çdo gjë tjetër në botë. Me mirënjohje dhe shpresë, \xa0Fjoralba Ruça ——————————————————————————Dear friends and kind-hearted people, I am Fjoralba Ruça, a teacher and a proud mother (single mother) of two beloved children, 8-year-old Gentoni and 15-year-old Pegi. In March of this year, my life took a dramatic turn when I learned I had stomach cancer. After a difficult surgery in Tirana to remove my entire stomach, I am now in Ankara, Turkey. Being away from my children is a pain I cannot put into words.  Every day is a battle. Each chemotherapy session is a step towards healing, but also a painful reminder of the struggle I am facing. I have completed 5 sessions out of two planned cycles, and I must continue on this path to have a chance of returning to Gentoni and Pegi, who are eagerly waiting for me at home. The support of friends and family has sustained me so far, but the expenses are significant and ongoing. Treatment, accommodation, and special dietary needs are essential but beyond my financial means. For this reason, I am reaching out to you with my heart in my hand. I know the world is full of people facing challenges, but if your heart feels moved to help me, I would be endlessly grateful. Any assistance you can provide is a precious gift that will bring me closer to my family and to hope for the future. Every bit of support is a ray of hope and a reminder of the love and solidarity that can create miracles. Please, join this fight, help me return to those who love me more than anything else in the world. \xa0With gratitude and hope, \xa0Fjoralba RuçaRead more"""

# Define the delimiter
delimiter = "——————————————————————————"

# Clean the text until the delimiter
cleaned_text = clean_text_until_delimiter(text[::-1], delimiter)

print(cleaned_text)


erom daeRaçuR ablarojF  ,epoh dna edutitarg htiW  .dlrow eht ni esle gnihtyna naht erom em evol ohw esoht ot nruter em pleh ,thgif siht nioj ,esaelP .selcarim etaerc nac taht ytiradilos dna evol eht fo rednimer a dna epoh fo yar a si troppus fo tib yrevE .erutuf eht rof epoh ot dna ylimaf ym ot resolc em gnirb lliw taht tfig suoicerp a si edivorp nac uoy ecnatsissa ynA .lufetarg ylsseldne eb dluow I ,em pleh ot devom sleef traeh ruoy fi tub ,segnellahc gnicaf elpoep fo lluf si dlrow eht wonk I .dnah ym ni traeh ym htiw uoy ot tuo gnihcaer ma I ,nosaer siht roF .snaem laicnanif ym dnoyeb tub laitnesse era sdeen yrateid laiceps dna ,noitadommocca ,tnemtaerT .gniogno dna tnacifingis era sesnepxe eht tub ,raf os em deniatsus sah ylimaf dna sdneirf fo troppus ehT .emoh ta em rof gnitiaw ylregae era ohw ,igeP dna inotneG ot gninruter fo ecnahc a evah ot htap siht no eunitnoc tsum I dna ,selcyc dennalp owt fo tuo snoisses 5 detelpmoc evah I .gnicaf ma I elggurts eht fo rednimer lufniap a osla

In [1]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import numpy as np

In [11]:
df=pd.read_csv('fundraiserLink.csv').iloc[:,1:]
df

,category,link
0,Medical,https://www.gofundme.com/f/support-the-furman-...
1,Medical,https://www.gofundme.com/f/bashke-kunder-kance...
2,Medical,https://www.gofundme.com/f/justin-riley-shaws-...
3,Medical,https://www.gofundme.com/f/help-vern-breathe-s...
4,Medical,https://www.gofundme.com/f/donate-to-rylans-jo...
...,...,...
17815,Wishes,https://www.gofundme.com/f/single-mother-seeki...
17816,Wishes,https://www.gofundme.com/f/birthday-wish-help-...
17817,Wishes,https://www.gofundme.com/f/support-leaura-and-...
17818,Wishes,https://www.gofundme.com/f/assist-jared-reyes-...


In [2]:
HEADERS=({'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/127.0.0.0 Safari/537.36'})

In [3]:
def bfsOverLinks(url):
    webpage=requests.get(url,headers=HEADERS)

    if webpage.status_code==200:
        soup=BeautifulSoup(webpage.content,"html.parser")

        amount=soup.find("div",attrs={'class':"progress-meter_progressMeterHeading__A6Slt"})
        amount = amount.text if amount else "not found"
        image=soup.find("img",attrs={'alt':"Main fundraiser photo"})
        image = image['src'] if image else "not found"
        story=soup.find("div",attrs={'class':'campaign-description_campaignDescription__6P_RU'}).text
        story = story.strip().replace('\n', ' ').replace('\r', ' ').replace('    ',' ').replace('   ',' ')
        story = story.text.strip() if story else "not found"
        timeline=soup.find("span",attrs={'class':"m-campaign-byline-created a-created-date"})
        timeline = timeline.text.strip() if timeline else "not found"

        return amount, image, story, timeline
    else:
        return "not found", "not found", "not found", "not found"

In [ ]:
df[['amount', 'image', 'story', 'timeline']] = df['link'].apply(lambda url: pd.Series(bfsOverLinks(url)))

In [ ]:
df.to_csv('updated_fundraiser.csv')

In [ ]:
# part-2

In [ ]:
fundraiser_df = pd.read_csv('updated_fundraiser.csv').iloc[:,1:].drop(columns=('link'))
fundraiser_df.sample(5)

,category,amount,image,story,timeline
12405,Faith,"$1,060 raised of $10,000 goal • 18 donations",https://images.gofundme.com/18FbLkDpT_oKqKwpdk...,"Donation protectedAssalaamu'alaikum,\n\n\n\nI ...","June 13th, 2024"
3119,Nonprofit,"$691 raised of $100,000 goal • 24 donations",https://images.gofundme.com/zhsj9lMJQ8TdAQ5PWW...,Tax deductibleSevere weather threats continue ...,"May 23rd, 2024"
14641,Sports,"$875 raised of $1,600 goal • 7 donations",https://images.gofundme.com/tpq4kO3-_UhYCpQqcm...,Donation protectedThis tough-as-nails 7-year-o...,"July 22nd, 2024"
9539,Competition,"$80 raised of $30,000 goal • 6 donations",https://images.gofundme.com/mm4hNwNGiYCIPP76AH...,"Donation protectedHello friends, family, and f...","July 17th, 2024"
13576,Family,"$40,905 raised of $50,000 goal • 389 donations",https://images.gofundme.com/IkMGYQ7UMlU28dokaF...,Donation protectedWe are devastated to share t...,"June 24th, 2024"


In [ ]:
print((fundraiser_df['amount']=="not found").value_counts())
print((fundraiser_df['image']=="not found").value_counts())
print((fundraiser_df['story']=="not found").value_counts())
print((fundraiser_df['timeline']=="not found").value_counts())

amount
False    17795
True        25
Name: count, dtype: int64
image
False    17437
True       383
Name: count, dtype: int64
story
False    17815
True         5
Name: count, dtype: int64
timeline
False    17815
True         5
Name: count, dtype: int64


In [ ]:
fundraiser_df['amount'] = fundraiser_df['amount'].where(fundraiser_df['amount'] != "not found", np.nan)
fundraiser_df['image'] = fundraiser_df['image'].where(fundraiser_df['image'] != "not found", np.nan)
fundraiser_df['story'] = fundraiser_df['story'].where(fundraiser_df['story'] != "not found", np.nan)
fundraiser_df['timeline'] = fundraiser_df['timeline'].where(fundraiser_df['timeline'] != "not found", np.nan)

In [ ]:
fundraiser_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17820 entries, 0 to 17819
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   category  17820 non-null  object
 1   amount    17795 non-null  object
 2   image     17437 non-null  object
 3   story     17815 non-null  object
 4   timeline  17815 non-null  object
dtypes: object(5)
memory usage: 696.2+ KB


### Amount preprocessing into raised amount and demanded amount

In [ ]:
import re
pattern = r"\$([0-9]{1,3}(?:,[0-9]{3})*)"
def reOverAmount(text):
    if pd.isna(text): 
        return np.nan, np.nan
    text=str(text)
    x=re.findall(pattern,text)# x will be list
    if len(x)==1:# if their is no fund raised then "$3,500 goal"
        return '0',x[0]
    elif len(x)==2:
        return x[0],x[1]
    else:
        return np.nan,np.nan
    

In [ ]:
fundraiser_df[['raised_amount($)','requested_amount($)']] = fundraiser_df['amount'].apply(lambda text:reOverAmount(text)).apply(pd.Series)

In [ ]:
fundraiser_df.drop(columns=('amount'),inplace=True)

In [ ]:
fundraiser_df.sample(5)

,category,image,story,timeline,raised_amount($),requested_amount($)
1367,Memorial,https://images.gofundme.com/N2rIvX2zPsTO0HpPdK...,Donation protectedHello fellow friends and lov...,"August 9th, 2024","9,688","10,000"
12380,Faith,https://images.gofundme.com/2hTLSp4eqaXAwSxTys...,Donation protectedHello Dance To Glory friends...,"July 10th, 2024","1,000","1,000"
1874,Memorial,https://images.gofundme.com/KUK9uOYSxwueaElEKj...,Donation protectedAs everyone knows my deddy h...,"August 5th, 2024","3,570","10,000"
14530,Sports,https://images.gofundme.com/kCp2TWKZ3KtoQIQ1Fl...,Donation protectedKenai has worked very hard t...,"June 27th, 2024",550,"5,000"
3380,Nonprofit,https://images.gofundme.com/fhpuD5V1YbzeCAMpC5...,Tax deductibleYou can make a difference by don...,Yesterday,0,"1,000"


In [ ]:
fundraiser_df.dropna(inplace=True)

In [ ]:
fundraiser_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 17417 entries, 0 to 17819
Data columns (total 6 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   category             17417 non-null  object
 1   image                17417 non-null  object
 2   story                17417 non-null  object
 3   timeline             17417 non-null  object
 4   raised_amount($)     17417 non-null  object
 5   requested_amount($)  17417 non-null  object
dtypes: object(6)
memory usage: 952.5+ KB


In [ ]:
fundraiser_df['timeline'].sample(5)
# '19 May 2024' this format is also present
# Today is present
# Yesterday is present

3825       May 30th, 2024
13142    August 5th, 2024
11622      June 8th, 2024
11691      May 29th, 2024
2386            Yesterday
Name: timeline, dtype: object

In [ ]:
from datetime import datetime,timedelta

In [ ]:
target_date = datetime(2024, 8, 14) #date of data extraction

def convert_to_datetime(date_str, reference_date):
    # Handle special cases
    if date_str == 'Today':
        return reference_date
    elif date_str == 'Yesterday':
        return reference_date - timedelta(days=1)
    
    # Remove ordinal suffixes and handle different formats
    date_str = re.sub(r'(\d+)(st|nd|rd|th)', r'\1', date_str)
    
    # Define formats
    formats = [
        '%B %d, %Y',    # e.g., August 5, 2024
        '%d %B %Y'      # e.g., 19 May 2024
    ]
    
    for fmt in formats:
        try:
            return datetime.strptime(date_str, fmt)
        except ValueError:
            continue
    
    return pd.NaT  # Return Not a Time if parsing fails

In [ ]:
# Apply conversion and calculate days
fundraiser_df['date_of_application'] = fundraiser_df['timeline'].apply(lambda x: convert_to_datetime(x, target_date))
fundraiser_df['days_of_fund_raising'] = fundraiser_df['date_of_application'].apply(lambda x: (target_date - x).days if pd.notna(x) else pd.NA)

In [ ]:
df_final=fundraiser_df.drop(columns=('timeline'))
df_final

,category,image,story,raised_amount($),requested_amount($),date_of_application,days_of_fund_raising
0,Medical,https://images.gofundme.com/HCcZpJ2vF8Oupas9rb...,"Donation protectedOn August 9th, Emil Furman, ...","36,554","5,000",2024-08-13,1
1,Medical,https://images.gofundme.com/-q_0SMy9Mmmvt--9-E...,Donation protectedFaleminderit per zemërgjerës...,0,"22,725",2024-08-11,3
2,Medical,https://images.gofundme.com/9I5n9sZwDdOMhrC4SG...,Donation protectedJustin “Riley” Shaw - a 26-y...,"42,122","75,000",2024-08-13,1
3,Medical,https://images.gofundme.com/ZxpdNzhwgMeI41rm1e...,Donation protectedMy uncle Vern has been in an...,"14,507","27,000",2024-07-24,21
4,Medical,https://images.gofundme.com/-iM30RBCFDg8nFloo3...,Donation protectedRylan is a beautiful and viv...,"34,379","50,000",2024-08-11,3
...,...,...,...,...,...,...,...
17815,Wishes,https://images.gofundme.com/U7KzmndtOMMldn6iJR...,Donation protectedAs a single mom and dedicate...,200,"10,000",2024-06-27,48
17816,Wishes,https://images.gofundme.com/7Q_w33NO4ciKwje7k_...,Donation protectedHelp us fill a birthday wish...,245,"3,000",2024-06-26,49
17817,Wishes,https://images.gofundme.com/AlK7wLVZ6WPU7gKULn...,Donation protectedMy name is LeAura. My daught...,570,"1,600",2024-06-26,49
17818,Wishes,https://images.gofundme.com/5wbdKeFq2wiL0b5pA4...,"Donation protectedDear Friends,\n\n\n\nI am wr...",70,"5,000",2024-06-28,47
